In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup as bs
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import fasterrcnn_resnet50_fpn, FasterRCNN_ResNet50_FPN_Weights, fasterrcnn_resnet50_fpn_v2
from torchvision import ops
from tqdm.notebook import tqdm
import os
import pickle
import math
from torchmetrics.detection.mean_ap import MeanAveragePrecision

In [2]:
# Connect to the GPU if one exists.
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print("Using: ", device)
torch.cuda.empty_cache()

Using:  cuda


In [12]:
def get_object_detection_model(model_path, version, num_classes = 2, feature_extraction = True, device = device):
    """
    Inputs
        num_classes: int
            Number of classes to predict. Must include the 
            background which is class 0 by definition!
        feature_extraction: bool
            Flag indicating whether to freeze the pre-trained 
            weights. If set to True the pre-trained weights will be  
            frozen and not be updated during.
    Returns
        model: FasterRCNN
    """
    if version == 2:
        model = fasterrcnn_resnet50_fpn_v2(weights='DEFAULT')
    elif version == 1:
        model = fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1)    
    # If True, the pre-trained weights will be frozen.
    if feature_extraction == True:
        for p in model.parameters():
            p.requires_grad = False    
    # Replace the original 91 class top layer with a new layer
    # tailored for num_classes.
    in_feats = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_feats, num_classes)
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    model.eval()
    return model

def detach_pred(prediction):
    boxes = prediction["boxes"]
    scores = prediction["scores"]
    labels = prediction["labels"]    
    return {"boxes":boxes.detach().cpu(), "labels":labels.detach().cpu(), "scores":scores.detach().cpu()}

def predict(model, image, device="cpu", transform = transforms.Compose([transforms.ToTensor()])):
    image = transform(np.array(image)).unsqueeze(0).type(torch.FloatTensor).to(device)
    pred = detach_pred(model(image)[0])
    return pred

def decode_prediction(prediction, 
                      score_threshold = 0.8, 
                      nms_iou_threshold = 0.1):
    """
    Inputs
        prediction: dict
        score_threshold: float
        nms_iou_threshold: float
    Returns
        prediction: tuple
    """
    boxes = prediction["boxes"]
    scores = prediction["scores"]
    labels = prediction["labels"]    
    # Remove any low-score predictions.
    if score_threshold is not None:
        want = scores > score_threshold
        boxes = boxes[want]
        scores = scores[want]
        labels = labels[want]    
    # Remove any overlapping bounding boxes using NMS.
    if nms_iou_threshold is not None:
        want = ops.nms(boxes = boxes, scores = scores, iou_threshold = nms_iou_threshold)
        boxes = boxes[want]
        scores = scores[want]
        labels = labels[want]    
    return {"boxes":boxes, "labels":labels, "scores":scores}



def get_seal_preds(sub_images, rcnn, device = device):
    preds = []
    for image in tqdm(sub_images):
        pred = predict(rcnn, image[0], device)
        preds.append(pred)
    return preds

def get_data(img_data, bb_data):
    seal_images = []
    seal_target = []
    for i in range(len(bb_data)):
        if bb_data[i] is not None:
            seal_box = []
            seal_label = []
            for row in range(bb_data[i].shape[0]):
                the_row = bb_data[i].iloc[row, :]
                new_box = []
                if int(the_row["xmin"]) != int(the_row["xmax"]) and int(the_row["ymin"]) != int(the_row["ymax"]):
                    new_box.append(int(the_row["xmin"]))
                    new_box.append(int(the_row["ymin"]))
                    new_box.append(int(the_row["xmax"]))
                    new_box.append(int(the_row["ymax"]))
                    seal_box.append(new_box)
                    seal_label.append(1)
            if len(seal_box) > 0:
                seal_images.append(img_data[i])
                new_target = {}
                new_target["boxes"] = torch.tensor(seal_box)
                new_target["labels"] = torch.tensor(seal_label)
                seal_target.append(new_target)
    seal_images = np.array(seal_images)
    return seal_images, seal_target



In [4]:
# Load in Data
train_img_data = np.load(r"C:\Users\kaanan\Desktop\Noses\train_images.npy", allow_pickle = True)
train_bb_data = np.load(r"C:\Users\kaanan\Desktop\Noses\train_bb_data.npy", allow_pickle = True)
train_images, train_targets = get_data(train_img_data, train_bb_data)

In [5]:
def compute_map(preds, score_thresh, nms_thresh, targets, device = device):
    preds = [decode_prediction(pred, score_thresh, nms_thresh) for pred in preds]
    metric = MeanAveragePrecision()
    metric.update(preds, targets)
    return metric.compute()

In [6]:
# Get Model 
rcnn_path = r"C:\Users\kaanan\Desktop\RCNN\RCNN Models\Additional Data\rcnn_extra_data_30"
rcnn = get_object_detection_model(rcnn_path, 2)

In [7]:
rcnn_less_data = r"C:\Users\kaanan\Desktop\RCNN\RCNN Evaluation Information\Trial 2\Models\rcnn_trial2_50"
rcnn_less = get_object_detection_model(rcnn_less_data, 2)

In [68]:
new_rcnn_path = r"C:\Users\kaanan\Desktop\RCNN\RCNN Models\Additional Data\rcnn_extra_data_v2_5"
new_rcnn = get_object_detection_model(new_rcnn_path, 1)

In [73]:
new_rcnn_more_path = r"C:\Users\kaanan\Desktop\RCNN\RCNN Models\Additional Data\rcnn_extra_data_v2_more_5"
new_rcnn_more = get_object_detection_model(new_rcnn_path, 1)

In [13]:
rcnn_preds = get_seal_preds(train_images, rcnn, device)
rcnn_less_preds = get_seal_preds(train_images, rcnn_less, device)

  0%|          | 0/11199 [00:00<?, ?it/s]

  0%|          | 0/11199 [00:00<?, ?it/s]

In [69]:
new_rcnn_preds = get_seal_preds(train_images, new_rcnn, device)

  0%|          | 0/11199 [00:00<?, ?it/s]

In [74]:
new_rcnn_more_preds = get_seal_preds(train_images, new_rcnn_more, device)

  0%|          | 0/11199 [00:00<?, ?it/s]

In [63]:
def average_iou(preds, targets):
    avg_iou = 0
    for i in range(len(preds)):
        ious = ops.box_iou(targets[i]["boxes"], preds[i]["boxes"]).numpy()
        if ious.shape[1] > 0:
            avg_iou += ious.max(axis = 1).mean()
    return avg_iou / len(preds)

In [70]:
average_iou(new_rcnn_preds, train_targets)

0.0037890584486934215

In [64]:
average_iou(rcnn_preds, train_targets)

0.01818738097770744

In [65]:
average_iou(rcnn_less_preds, train_targets)

0.03553283928556558

In [67]:
average_iou(rcnn_preds_cleaned, train_targets)

0.0007192998406463572

In [14]:
compute_map(rcnn_preds, 0, .2, train_targets, device)

{'map': tensor(2.1645e-05),
 'map_50': tensor(4.8298e-05),
 'map_75': tensor(1.5671e-05),
 'map_small': tensor(2.1645e-05),
 'map_medium': tensor(0.),
 'map_large': tensor(0.),
 'mar_1': tensor(0.0004),
 'mar_10': tensor(0.0008),
 'mar_100': tensor(0.0008),
 'mar_small': tensor(0.0019),
 'mar_medium': tensor(0.),
 'mar_large': tensor(0.),
 'map_per_class': tensor(-1.),
 'mar_100_per_class': tensor(-1.)}

In [15]:
compute_map(rcnn_less_preds, 0, .2, train_targets, device)

{'map': tensor(2.2641e-05),
 'map_50': tensor(5.8287e-05),
 'map_75': tensor(1.0391e-05),
 'map_small': tensor(2.2641e-05),
 'map_medium': tensor(0.),
 'map_large': tensor(0.),
 'mar_1': tensor(0.0005),
 'mar_10': tensor(0.0012),
 'mar_100': tensor(0.0012),
 'mar_small': tensor(0.0028),
 'mar_medium': tensor(0.),
 'mar_large': tensor(0.),
 'map_per_class': tensor(-1.),
 'mar_100_per_class': tensor(-1.)}

In [71]:
compute_map(new_rcnn_preds, 0, .2, train_targets, device)

{'map': tensor(6.5107e-06),
 'map_50': tensor(3.1633e-05),
 'map_75': tensor(0.),
 'map_small': tensor(6.5107e-06),
 'map_medium': tensor(0.),
 'map_large': tensor(0.),
 'mar_1': tensor(1.6794e-05),
 'mar_10': tensor(3.9185e-05),
 'mar_100': tensor(3.9185e-05),
 'mar_small': tensor(8.7353e-05),
 'mar_medium': tensor(0.),
 'mar_large': tensor(0.),
 'map_per_class': tensor(-1.),
 'mar_100_per_class': tensor(-1.)}

In [16]:
compute_map(rcnn_preds, 0, .05, train_targets, device)

{'map': tensor(2.2446e-05),
 'map_50': tensor(4.9314e-05),
 'map_75': tensor(1.6540e-05),
 'map_small': tensor(2.2446e-05),
 'map_medium': tensor(0.),
 'map_large': tensor(0.),
 'mar_1': tensor(0.0004),
 'mar_10': tensor(0.0006),
 'mar_100': tensor(0.0006),
 'mar_small': tensor(0.0014),
 'mar_medium': tensor(0.),
 'mar_large': tensor(0.),
 'map_per_class': tensor(-1.),
 'mar_100_per_class': tensor(-1.)}

In [17]:
compute_map(rcnn_less_preds, 0, .05, train_targets, device)

{'map': tensor(2.2612e-05),
 'map_50': tensor(5.6431e-05),
 'map_75': tensor(1.0713e-05),
 'map_small': tensor(2.2612e-05),
 'map_medium': tensor(0.),
 'map_large': tensor(0.),
 'mar_1': tensor(0.0005),
 'mar_10': tensor(0.0007),
 'mar_100': tensor(0.0007),
 'mar_small': tensor(0.0017),
 'mar_medium': tensor(0.),
 'mar_large': tensor(0.),
 'map_per_class': tensor(-1.),
 'mar_100_per_class': tensor(-1.)}

In [72]:
compute_map(new_rcnn_preds, 0, .05, train_targets, device)

{'map': tensor(6.7204e-06),
 'map_50': tensor(3.2533e-05),
 'map_75': tensor(0.),
 'map_small': tensor(6.7204e-06),
 'map_medium': tensor(0.),
 'map_large': tensor(0.),
 'mar_1': tensor(1.6794e-05),
 'mar_10': tensor(3.9185e-05),
 'mar_100': tensor(3.9185e-05),
 'mar_small': tensor(8.7353e-05),
 'mar_medium': tensor(0.),
 'mar_large': tensor(0.),
 'map_per_class': tensor(-1.),
 'mar_100_per_class': tensor(-1.)}